In [50]:
import re
import csv
from time import sleep
from bs4 import BeautifulSoup
from bs4 import * 
import requests

headers = {
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'referer': 'https://www.google.com',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36 Edg/85.0.564.44'
}

def get_article(card):
    """Extract article information from the raw html"""
    headline = card.find('div', style="width:550px").strong.text
    posted = card.find('p', 'PT3').text.replace('\xa0|\xa0  Source: Moneycontrol.com', '').strip()
    
    article = (headline, posted)
    return article

def get_the_news(search):
    """Run the main program"""
    template = 'https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=RI&durationType=Y&Year={}'
    url = template.format(search)
    articles = []
    
    while True:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'MT15')
        
        # extract articles from page
        for card in cards:
            article = get_article(card)
            articles.append(article)
                    
                
        # find the next page
        try:
            url = soup.find('div', 'MR10').a.get('href')
            sleep(1)
        except AttributeError:
            break
            
    # save article data
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Headline', 'Posted'])
        writer.writerows(articles)
        
    return articles

In [51]:
articles = get_the_news('2020')

AttributeError: ignored